# Build Simple Decision Tree Using CART Algorithm
There are several types of decision trees, here I will build a tree from scratch using the CART algorithm. CART stands for Classification And Regression Tree. I first read this algorithm on [this blog](https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/), and determined to try it by myself. I basically followed the same idea, but instead of using dictionaries to store nodes, I will use class to define a node as well as the tree.

## Build a tree node
The decision tree is made of several levels of tree nodes, and each node has 2 childs. For leaf nodes, there will be a terminal value that returns as the prediction of classification or regression. If a node only contains one label, then it will automatically becames a leaf node. For other intermediate nodes, there will be index and value, which specify which feature column to compare and the cutoff value for splitting. The index and value are selected by looping over all possible indexes and values in training set, and find the best split that gives the lowest gini index.

In [1]:
class Node(object):
    def __init__(self, groups):
        '''Initialized a node, it has left child and right child, cutoff value,
        index for feature column, and terminal value if it's a leaf node.
        If a node only has one label, make it a leaf node.
        
        Input:
            groups: a list of examples
        '''
        self.groups = groups
        self.left = None
        self.right = None
        self.value = None
        self.index = None
        self.terminal = None
        if len(Node.get_labels(groups)) == 1:
            self.to_terminal()
    def get_size(self):
        '''Return the size of examples in current node'''
        return len(self.groups)
    @staticmethod
    def get_labels(groups):
        '''Return a list of unique labels'''
        return list(set([item[-1] for item in groups]))
    @staticmethod
    def gini(groups):
        '''Calculate the gini index of current node
        gini = 1-sum(p(i)**2), i in all labels
        '''
        if not groups:
            return 0
        gini = 1
        for label in Node.get_labels(groups):
            gini -= ([item[-1] for item in groups].count(label)/len(groups))**2
        return gini
    def split(self):
        '''Try different splits and pick the split with the smallest gini index'''
        gini = 1
        # O(n^2), can we do better?
        for index in range(len(self.groups[0])-1):
            for value in [item[index] for item in self.groups]:
                leftlist, rightlist = self.split_groups(index, value)
                gini_c = (len(leftlist)*Node.gini(leftlist)+len(rightlist)*Node.gini(rightlist))/len(self.groups)
                if gini_c < gini:
                    gini = gini_c
                    self.value = value
                    self.index = index
                    # this may creates a lot of unused Node objects
                    self.left = Node(leftlist)
                    self.right = Node(rightlist)
    def split_groups(self, index, value):
        '''Split the group by the index and value
        
        Input:
            index: feature column
            value: cutoff value; example with feature column smaller than this will
        assigned to left child, otherwise right child.
        
        Return:
            list of examples for left child and right child
        '''
        leftlist, rightlist = [], []
        for example in self.groups:
            if example[index] < value:
                leftlist.append(example)
            else:
                rightlist.append(example)
        return leftlist, rightlist
    def to_terminal(self):
        '''Save this node as terminal leaf
        '''
        labels = Node.get_labels(self.groups)
        # might have a problem when the counts are the same
        self.terminal = max(labels, key = labels.count)
    def get_ind_val(self):
        '''Return the index and value'''
        return self.index, self.value
    def get_terminal(self):
        '''Return the classification result'''
        return self.terminal

## Will some of the leaves have no terminal values?
When I first implemented the code in the previous cell, I was somehow afraid that some leaf nodes won't have terminal values, and may return None if we want to retrieve the terminal value. After a second thought, I now know that it won't happen as long as the min_size is set to 1 or higher. Take the binary case for example. 
If a node has only one group, then it's a leaf node and won't split again, and the terminal value is just the label of the group.

If a parent node has n (n &ge; 2) groups, then it must contains mixed labels, otherwise it's a leaf node and won't have childs. And we will prove that both of its childs are not be None. That's because if one of its childs is None, then the gini index of its two childs is the same with the parent node. Intuitively, the gini index can be reduced by just seperating one positive group from the groups. And this could be proven by following:

*Let n be the total number of groups in current node, and x be the positive labels (1 &le; x < n).*

*The gini index of current node: $gini_0 = 1-\big(\frac{x}{n}\big)^2+\big(\frac{n-x}{n}\big)^2$*

*Split one positive label, and the gini index now is $gini_1 = \frac{n-1}{n}\bigg(1-\big(\frac{x-1}{n-1}\big)^2+\big(\frac{n-x}{n-1}\big)^2\bigg)$*

*Substract the two equations and after several steps of algebra, we will have:
$gini_0-gini_1 = \frac{2(n-x)^2}{n^2(n-1)} > 0$, which means spliting will be performed and the children will have at least one group.*

Combine these 2 cases together, we know that the leaf node will have at least one group, hence will have a terminal value.

## Build a decision tree
A decision tree has a root node, and by training the dataset, we will add nodes to it childs. This process is done by recursively splitting the training sets, until the tree reaches the maximum depth, or all its childs are leaf node.

For prediction, it's similar. Recursively assigning the camparison of features to nodes from top to bottom until it finds a leaf node and return a terminal value.

In [2]:
class CART_tree(object):
    def __init__(self):
        '''Initialize a tree with root node, max_depth and min_size'''
        self.root = None
        self.max_depth = None
        self.min_size = None
    def train(self, training, max_depth = 1, min_size = 1):
        '''Train the tree with training examples
        
        Input:
            max_depth, int
            min_size, int
        '''
        self.root = Node(training)
        self.max_depth = max_depth
        self.min_size = min_size
        self.rec_split(self.root, 0)
    def rec_split(self, node, depth):
        '''Recursively split the node, until meet either of the following conditions:
        1. Max tree depth or minimum node size reached.
        2. The node is leaf node that only has one label.
        
        Input:
            node: Node object
            depth: int
        '''
        if depth == self.max_depth or node.get_size() <= self.min_size:
            # reach the max_depth or min_size, return
            node.to_terminal()
            return
        if node.get_terminal() is not None:
            # leaf node that only contains one label, return
            return
        # after previous 2 cases, the remaining nodes have group size larger than 2
        # and have mixed labels
        node.split()
        self.rec_split(node.left, depth+1)
        self.rec_split(node.right, depth+1)
        return
    def predict(self, groups):
        '''Given a list of groups, predict their labels
        
        Input:
            groups: list, input features
        Return:
            preds: list, predicted labels
        '''
        preds = []
        for group in groups:
            preds.append(CART_tree._predict(group, self.root))
        return preds
    @staticmethod
    def _predict(group, node):
        '''Recursively assign the input feature to one of the childs of 
        current node, until it finds a leaf node.
        
        Input:
            group: list, input feature
            node: Node object
        Return:
            terminal value of a leaf node
        '''
        if node.get_terminal() is not None:
            return node.get_terminal()
        index, value = node.get_ind_val()
        if group[index] < value:
            return CART_tree._predict(group, node.left)
        else:
            return CART_tree._predict(group, node.right)

## Test this with a simple example

In [3]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

In [4]:
temp = CART_tree()
temp.train(dataset, 1, 1)
temp.predict(dataset)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

## More to do
1. Use entropy to split nodes
1. Tree pruning to reduce overfitting
1. Categorical dataset, use equality instead of ranking
1. Regression, use different cost function and method to create leaf nodes